# Binary Sentiment Analysis of French Movie Reviews

### Objectives
1. Text cleaning
2. Text preprocessing for custom embedding Neural Network
3. Train RNN model for sentiment analysis

⚠️ This notebook will be your final deliverable. 
- Make sure it can run "restart and run all"
- Delete useless code cells
- Do not "clear output"

# 0. Load data

Our dataset contains 30,000 french reviews of movies, along with the binary class 1 (positive) or 0 (negative) score

In [90]:
import pandas as pd
import numpy as np

In [44]:
# We load the dataset for you
data = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies.csv')
data

,review,polarity
0,Ça commence à devenir énervant d'avoir l'impre...,0
1,"J'ai aimé ce film, si il ressemble a un docume...",1
2,Une grosse merde ce haneke ce faire produire p...,0
3,"Beau mélodrame magnifiquement photographié, ""V...",1
4,A la poursuite du diamant vers est un film pro...,1
...,...,...
29946,Le meilleur film de super-héros derrière le ba...,1
29947,Un drame qui est d'une efficacité remarquable....,1
29948,"Une daube hollywoodienne de plus, aucun intérê...",0
29949,Et voilà un nouveau biopic sur la star du X Li...,0


In [45]:
# We create features
y = data.polarity
X = data.review

# We analyse class balance
print(pd.value_counts(y))

1    15051
0    14900
Name: polarity, dtype: int64


In [46]:
# We check various reviews
print(f'polarity: {y[0]} \n')
print(X[0])

polarity: 0 

Ça commence à devenir énervant d'avoir l'impression de voir et revoir le même genre de film à savoir : la comédie romantique, surement le genre le plus prolifique de le production française actuelle. Le problème c'est que l'on a souvent affaire à des niaiseries de faible niveau comme celui ci. Avec un scenario ultra balisé et conventionnel, c'est à se demander comment ça peut passer les portes d'un producteur. Bref cette sempiternel histoire d'un homme mentant au nom de l'amour pour reconquérir une femme et qui à la fin se prend son mensonge en pleine figure est d'une originalité affligeante, et ce n'est pas la présence au casting de l'ex miss météo Charlotte Le Bon qui rêve surement d'avoir la même carrière que Louise Bourgoin qui change la donne.


# 1. Clean Text

❓ We need to give a _quick & dirty_ cleaning to all the sentences in the dataset. Create a variable `X_clean` of similar shape, but with the following cleaning:
- Replace french accents by their non-accentuated equivalent using the [unidecode.unidecode()](https://pypi.org/project/Unidecode/) method
- Reduce all uppercases to lowercases
- Remove any characters outside of a-z, for instance using `string.isalpha()`

😌 You will be given the solution `X_clean` in the next question to make sure you can complete the challenge

In [47]:
### YOUR CODE
X_clean=X

In [48]:
! pip install unidecode
from unidecode import unidecode

In [49]:
#removing accents:
X_clean=X_clean.apply(lambda x: unidecode(x))

#lowering text
X_clean=X_clean.apply(lambda x: x.lower())

#removing punctuation
import string 

def punctuation_removal(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

X_clean=X_clean.apply(lambda x: punctuation_removal(x))    

In [50]:
Xtest=X_clean

In [41]:
from nbresult import ChallengeResult

result = ChallengeResult('C14',
    shape = X_clean.shape,
    first_sentence = X_clean[0]
)
result.write()

# 2. Preprocess data

Now that we have clean sentences, we need to convert each one into a list of integers of fixed size
- For example, the sentence: `"this was good"` should become something like `array([1, 3, 18, 0, 0, 0, ...0], dtype=int32)` where each integer match to a each _unique_ word in your corpus of sentences.

❓ Create a numpy ndarray `X_input` of shape (29951, 100) that will be the direct input to your Neutral Network. 

- 29951 represents the number of reviews in the dataset `X_clean`
- 100 represents the maximum number of words to keep for each movie review.
- It must contain only numerical values, without any `NaN`
- In the process, compute and save the number of _unique_ words in your cleaned corpus under `vocab_size` variable

👉 First, you **must** start back from the clean solution below (14Mo)

In [122]:
X_clean = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_clean.csv")['review']
X_clean.shape

(29951,)

In [123]:
### YOUR CODE

#splitting sentences into words
def convert_sentences(X):
    return [sentence.split(' ') for sentence in X]

X_clean = convert_sentences(X_clean)

In [124]:
#creating a word to id dictionnary
word_to_id = {}
iter_ = 1
for sentence in X_clean:
    for word in sentence:
        if word in word_to_id:
            continue
        word_to_id[word] = iter_
        iter_ += 1
        

In [125]:
print(f'There are {len(word_to_id)} different words in the X_clean sentences')
vocab_size=len(word_to_id)

There are 62379 different words in the X_clean sentences


In [126]:
#creating a tokenize function to swap words by their number_id
def tokenize(sentences, word_to_id):
    return [[word_to_id[_] for _ in s if _ in word_to_id] for s in sentences]

In [127]:
#tokenizing
X_clean = tokenize(X_clean, word_to_id)

In [128]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#padding reviews in case they are les than 100 words long with 0 at the end
X_clean = pad_sequences(X_clean, dtype='float32', padding='post')


In [129]:
#padding worked
X_clean[2][:100]

array([ 59., 102., 103.,  72., 104.,  72., 105., 106., 100.,  28., 107.,
         8., 108., 109., 110., 111., 112., 113.,  25., 114., 115.,  25.,
        46., 116.,  72.,  25.,  94., 117., 113.,  25.,  46., 118., 119.,
       120., 121., 122.,  64., 123.,  69., 124.,  19., 125.,  25., 126.,
       127.,   6., 103.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.], dtype=float32)

In [130]:
X_clean.shape

(29951, 352)

In [82]:
# Now we need to keep 100 first words only for each review
X_clean_final=[]
for i in range(len(X_clean)):
    X_clean_final.append(X_clean[i][:100])

In [106]:
X_input=np.reshape(X_clean_final,(29951,100))
X_input=X_input.astype('int32')
X_input

array([[    1,     2,     3, ...,     6,    76,    77],
       [   86,    72,    13, ...,     0,     0,     0],
       [   59,   102,   103, ...,     0,     0,     0],
       ...,
       [   59,  2439,  5003, ...,     0,     0,     0],
       [    8,  1208,    36, ..., 41774, 62377,    10],
       [   36,    13,    61, ...,  3039,    25,    10]], dtype=int32)

In [107]:
from nbresult import ChallengeResult

result = ChallengeResult('C1415',
    type_X = type(X_input),
    shape = X_input.shape, 
    input_1 = X_input[1], 
)
result.write()

# 3. Neural Network

❓Create and fit a Neural Netork that takes `X_input` and `y` as input, to binary classify each sentence's sentiment

- You cannot use transfer learning or other pre-existing Word2Vec models
- You must use a "recurrent" architecture to _capture_ a notion of order in the sentences' words
- The performance metrics for this task is "accuracy"
- Store your model in a variable `model` 
- Store the result your `model.fit()` in a variable `history`. 
- ⚠️ `history.history` must comprises a measure of the `val_accuracy` at each epoch.
- You don't need to cross-validate your model

😌 Don't worry, you will not be judged on your computer power: You should be able to reach accuracy significantly better than baseline in less than 3 minutes even without GPUs.

👉 But first, you **must** start back from the solution below (70Mo)

In [101]:
url = 'https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_input.csv'
X_input = np.genfromtxt(url, delimiter=',', dtype='int32')

In [114]:
## YOUR CODE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_input, y, test_size=0.3)


In [116]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers 

def init_model(vocab_size):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size+1, output_dim=30, mask_zero=True))
    model.add(layers.LSTM(10))
    model.add(layers.Dense(5))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    return model

In [118]:
from tensorflow.keras.callbacks import EarlyStopping

model = init_model(len(word_to_id))

es = EarlyStopping(patience=5, restore_best_weights=True)

history=model.fit(X_train, y_train, 
          epochs=10, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/10
459/459 [==============================] - 82s 174ms/step - loss: 0.5594 - accuracy: 0.7049 - val_loss: 0.3121 - val_accuracy: 0.8760
Epoch 2/10
459/459 [==============================] - 81s 177ms/step - loss: 0.2311 - accuracy: 0.9114 - val_loss: 0.2861 - val_accuracy: 0.8852
Epoch 3/10
459/459 [==============================] - 78s 169ms/step - loss: 0.1527 - accuracy: 0.9431 - val_loss: 0.2978 - val_accuracy: 0.8819
Epoch 4/10
459/459 [==============================] - 86s 186ms/step - loss: 0.1251 - accuracy: 0.9548 - val_loss: 0.3289 - val_accuracy: 0.8785
Epoch 5/10
459/459 [==============================] - 79s 172ms/step - loss: 0.1043 - accuracy: 0.9613 - val_loss: 0.3718 - val_accuracy: 0.8773
Epoch 6/10
459/459 [==============================] - 72s 156ms/step - loss: 0.0890 - accuracy: 0.9690 - val_loss: 0.4045 - val_accuracy: 0.8744
Epoch 7/10
459/459 [==============================] - 74s 161ms/step - loss: 0.0778 - accuracy: 0.9716 - val_loss: 0.4096 - val_ac

In [120]:
history.history

{'loss': [0.4468790888786316,
  0.22659480571746826,
  0.16633781790733337,
  0.13486681878566742,
  0.11261698603630066,
  0.09949854761362076,
  0.0876273363828659],
 'accuracy': [0.7979556918144226,
  0.9123679995536804,
  0.9363543391227722,
  0.9496422410011292,
  0.9582964181900024,
  0.9644293189048767,
  0.9676320552825928],
 'val_loss': [0.3121359348297119,
  0.28613102436065674,
  0.29783540964126587,
  0.3289003372192383,
  0.37181761860847473,
  0.4045064449310303,
  0.4096420705318451],
 'val_accuracy': [0.8759936690330505,
  0.885214626789093,
  0.8818759918212891,
  0.8785373568534851,
  0.8772655129432678,
  0.8744038343429565,
  0.8739268779754639]}

In [121]:
from nbresult import ChallengeResult
result = ChallengeResult('C1517',
                         history=history.history)
result.write()